In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
DMRs = pd.read_table("DMR_annotation_beta.40.3_ratio.1.bed", header=None)
DMRs.index = DMRs[0]+'.'+DMRs[1].astype(str)+'.'+DMRs[2].astype(str)
DMRs

In [ ]:
pd.crosstab(DMRs[6],DMRs[7])

In [ ]:
inDMRs = {}
inDMRs2 = {}
for i in DMRs.index:
    for j in range(int(DMRs[1][i]), int(DMRs[2][i]+1)):
        inDMRs[j] = DMRs[6][i]
        inDMRs2[j] = DMRs[3][i]

In [ ]:
DMR_order = {
    '0,1,2|3,4':0, 
    '0|others':1, 
    '1|others':2, 
    '2|others':3, 
    '3|others':4, 
    '4|others':5,
    '0|1,2':6, 
    '3|4':7,
}

In [ ]:
beta = pd.read_table('beta.40.3_ratio.1.metilene')
beta['DMR'] = beta['pos'].map(inDMRs)
beta['DMRo'] = beta['DMR'].map(DMR_order)
beta['diff'] = beta['pos'].map(inDMRs2)
beta = beta.dropna().sort_values(['DMRo','diff'])
beta

In [ ]:
pd.crosstab(beta['DMR'],beta['DMRo'])

In [ ]:
dict_sample_color = {}
dict_group_color = {}
for i in range(7):
    dict_group_color[str(i)] = sns.color_palette("Set2")[i]
for i in beta.columns[2:-3]:
    dict_sample_color[i] = dict_group_color[i.split('_')[0]]

sns.color_palette("Set2")

In [ ]:
sns.clustermap(beta[beta.columns[2:-3]],method='ward',
               # row_cluster=False,
               col_colors=beta.columns[2:-3].map(dict_sample_color),
               cmap='coolwarm', figsize=[5,5], xticklabels=False, yticklabels=False)

In [ ]:
for i in DMR_order.keys():
    print(i+'(N='+str(sum(DMRs[6]==i))+')') 
    cm = sns.clustermap(beta[beta['DMR']==i][beta.columns[2:-3]],
                   method='ward',
                   col_colors=beta.columns[2:-3].map(dict_sample_color),
                   cmap='coolwarm', figsize=[3,3], 
                   xticklabels=False, yticklabels=False)
    cm.cax.set_visible(False)
    cm.fig.suptitle(i+'(N='+str(sum(DMRs[6]==i))+')') 

In [ ]:
Predictions = pd.read_table("prediction.res", encoding='iso-8859-1', skiprows=90)
Predictions_anno = pd.read_table("prediction.res", encoding='iso-8859-1', sep='@', header=None).head(90)

In [ ]:
Predictions.sort_values('start').tail()

In [ ]:
Predictions.index = Predictions['chr']+'.'+\
                    Predictions['start'].astype(str)+'.'+\
                    Predictions['stop'].astype(str)
df_res = pd.merge(DMRs, Predictions, left_index=True, right_index=True, how='outer').fillna('NA')
cmp = pd.crosstab(df_res[6], df_res['sig.comparison'])[pd.crosstab(df_res[6], df_res['sig.comparison']).T.idxmax()[:-1]]
cmp['simulated'] = cmp.index.map(pd.crosstab(DMRs[6],DMRs[7]).T.sum())
cmp

In [ ]:
for i in pd.crosstab(df_res[6], df_res['sig.comparison']).T.idxmax()[:-1]:
    print(Predictions_anno.loc[i][0])

In [ ]:
for i in pd.crosstab(df_res[6], df_res['sig.comparison']).T.sort_values('3|4', ascending=False).index[:20]:
    try:
        print(pd.crosstab(df_res[6], df_res['sig.comparison']).T['3|4'][i], Predictions_anno.loc[i][0])
    except:
        pass